In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cython
import timeit
import math

In [57]:
%load_ext cython

Python is slow because it has to do a lot of guessing. There is no typing of variables, so, for example, adding `a + b` could be addition, string concatenation, list concatenation, etc.

# Native code compilation

We will see how to convert Python code to native compiled code. We will use the example of calculating the pairwise distance between a set of vectors, a $O(n^2)$ operation. 

For native code compilation, it is usually preferable to use explicit for loops and minimize the use of `numpy` vectorization and broadcasting because

- It makes it easier for the `numba` JIT to optimize
- It is easier to "cythonize"
- It is easier to port to C++

However, use of vectors and matrices is fine especially if you will be porting to use a C++ library such as Eigen.

## Timing code

### Manual

In [2]:
import time

def f(n=1):
    start = time.time()
    time.sleep(n)
    elapsed = time.time() - start
    return elapsed

In [3]:
f(1)

1.00107741355896

### Clock time

The `time` magic function calls the Unix `time` command. This returns 3 different times:

- user time is time spent by user code and libraries
- sys time is time spent on operating system calls (kernel calls)
- wall time is time that has elapsed from your perspective

For concurrent programs, user/sys time can be greater than wall time.

In [4]:
%%time

time.sleep(1)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1 s


### Using `timeit`

The `-r` argument says how many runs to average over, and `-n` says how many times to run the function in a loop per run. See `%timeit?` for more information.

In [5]:
%timeit time.sleep(0.01)

10.1 ms ± 6.14 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit -r3 time.sleep(0.01)

10.1 ms ± 8.89 µs per loop (mean ± std. dev. of 3 runs, 100 loops each)


In [7]:
%timeit -n10 time.sleep(0.01)

10.1 ms ± 25 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit -r3 -n10 time.sleep(0.01)

10.1 ms ± 10.1 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


The `-o` flag returns an object of the time statistics

In [9]:
t1 = %timeit -n10 -o time.sleep(0.01)

10.1 ms ± 7.76 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
t1

<TimeitResult : 10.1 ms ± 7.76 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)>

In [11]:
', '.join([method for method in dir(t1) if not method.startswith('_')])

'all_runs, average, best, compile_time, loops, repeat, stdev, timings, worst'

You can also use `timeit` as a Python module.

Pass it a callable with no arguments or a string. This is not as convenient as the magic function. 

In [12]:
import timeit

In [13]:
timeit.timeit('time.sleep(0.01)', number=10)

0.10094802593812346

In [14]:
timeit.timeit(lambda: time.sleep(0.01), number=10)

0.10125513887032866

### Time unit conversions

```
1 s = 1,000 ms
1 ms = 1,000 µs
1 µs = 1,000 ns
```

## Profiling

If you want to identify bottlenecks in a Python script, do the following:
    
- First make sure that the script is modular - i.e. it consists mainly of function calls
- Each function should be fairly small and only do one thing
- Then run a profiler to identify the bottleneck function(s) and optimize them

See the Python docs on [profiling Python code](https://docs.python.org/3/library/profile.html)

Profiling can be done in a notebook with %prun, with the following readouts as column headers:

- ncalls
    - for the number of calls,
- tottime
    - for the total time spent in the given function (and excluding time made in calls to sub-functions),
- percall
    - is the quotient of tottime divided by ncalls
- cumtime
    - is the total time spent in this and all subfunctions (from invocation till exit). This figure is accurate even for recursive functions.
- percall
    - is the quotient of cumtime divided by primitive calls
- filename:lineno(function)
    - provides the respective data of each function 
    
See `%prun?` for more information.

In [15]:
def foo1(n):
    return np.sum(np.square(np.arange(n)))

def foo2(n):
    return sum(i*i for i in range(n))

def foo3(n):
    [foo1(n) for i in range(10)]
    foo2(n)

def foo4(n):
    return [foo2(n) for i in range(100)]
    
def work(n):
    foo1(n)
    foo2(n)
    foo3(n)
    foo4(n)

In [18]:
%%time

work(int(1e5))

CPU times: user 996 ms, sys: 4 ms, total: 1 s
Wall time: 998 ms


- `-D` saves results in a form that the `pstats` moudle can parse.
- `-q` suppresses output

In [17]:
%prun -q -D work.prof work(int(1e5))

 
*** Profile stats marshalled to file 'work.prof'. 


In [19]:
import pstats
p = pstats.Stats('work.prof')
p.print_stats()
pass

Tue Feb 16 22:27:53 2021    work.prof

         10200435 function calls in 18.228 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   18.228   18.228 {built-in method builtins.exec}
       11    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
      102    8.941    0.088   18.216    0.179 {built-in method builtins.sum}
       11    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
       11    0.005    0.000    0.011    0.001 <ipython-input-15-32cd1fde8562>:1(foo1)
        1    0.000    0.000   17.853   17.853 <ipython-input-15-32cd1fde8562>:11(foo4)
        1    0.000    0.000   18.228   18.228 <string>:1(<module>)
        1    0.000    0.000    0.009    0.009 <ipython-input-15-32cd1fde8562>:8(<listcomp>)
        1    0.000    0.000    0.189    0.189 <ipython-input-15-32cd1fde8562>:7(foo3)
        1    0.000    0.000   18.228   18.228 <ipython-input-15-3

In [20]:
p.sort_stats('time', 'cumulative').print_stats('foo')
pass

Tue Feb 16 22:27:53 2021    work.prof

         10200435 function calls in 18.228 seconds

   Ordered by: internal time, cumulative time
   List reduced from 22 to 4 due to restriction <'foo'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       11    0.005    0.000    0.011    0.001 <ipython-input-15-32cd1fde8562>:1(foo1)
      102    0.001    0.000   18.217    0.179 <ipython-input-15-32cd1fde8562>:4(foo2)
        1    0.000    0.000    0.189    0.189 <ipython-input-15-32cd1fde8562>:7(foo3)
        1    0.000    0.000   17.853   17.853 <ipython-input-15-32cd1fde8562>:11(foo4)




In [21]:
p.sort_stats('ncalls').print_stats(5)
pass

Tue Feb 16 22:27:53 2021    work.prof

         10200435 function calls in 18.228 seconds

   Ordered by: call count
   List reduced from 22 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 10200102    9.274    0.000    9.274    0.000 <ipython-input-15-32cd1fde8562>:5(<genexpr>)
      102    8.941    0.088   18.216    0.179 {built-in method builtins.sum}
      102    0.001    0.000   18.217    0.179 <ipython-input-15-32cd1fde8562>:4(foo2)
       11    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
       11    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}




We can get the results object directly.

In [22]:
profile = %prun -r -q work(int(1e5))

In [23]:
profile.sort_stats('cumtime').print_stats(5)
pass

         10200435 function calls in 18.191 seconds

   Ordered by: cumulative time
   List reduced from 22 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   18.191   18.191 {built-in method builtins.exec}
        1    0.000    0.000   18.191   18.191 <string>:1(<module>)
        1    0.000    0.000   18.191   18.191 <ipython-input-15-32cd1fde8562>:14(work)
      102    0.001    0.000   18.179    0.178 <ipython-input-15-32cd1fde8562>:4(foo2)
      102    8.919    0.087   18.179    0.178 {built-in method builtins.sum}




We may not need `pstats` for simple analysis. This limits to calls with `foo` in the name, and sorts by `cumtime`.

In [25]:
%prun -l foo -s cumtime work(int(1e5))

## Optimizing a function

Our example will be to optimize a function that calculates the pairwise distance between a set of vectors.

We first use a built-in function from`scipy` to check that our answers are right and also to benchmark how our code compares in speed to an optimized compiled routine.

In [26]:
from scipy.spatial.distance import squareform, pdist

In [27]:
n = 100
p = 100
xs = np.random.random((n, p))

We save the result to compare with our own implementations later.

In [28]:
sol = squareform(pdist(xs))

In [29]:
%timeit -r3 -n3 squareform(pdist(xs))

361 µs ± 59.5 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Python

### Simple version

In [30]:
def pdist_py(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(p):
                A[i,j] += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(A[i,j])
    return A

Note that we 

- first check that the output is **right**
- then check how fast the code is

In [31]:
func = pdist_py
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
1.29 s ± 424 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Exploiting symmetry

In [32]:
def pdist_sym(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            for k in range(p):
                A[i,j] += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(A[i,j])
    A += A.T
    return A

`np.allclose()` does element-wise check of numpy arrays / matrices, allowing some very small tolerance for floating point comparison

In [33]:
func = pdist_sym
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
641 ms ± 823 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Vectorizing inner loop

In [34]:
def pdist_vec(xs): 
    """Vectorize inner loop."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            A[i,j] = np.sqrt(np.sum((xs[i] - xs[j])**2))
    A += A.T
    return A

In [35]:
func = pdist_vec
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
57.2 ms ± 388 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Broadcasting and vectorizing

Note that the broadcast version does twice as much work as it does not exploit symmetry.

In [36]:
def pdist_numpy(xs):
    """Fully vectroized version."""
    return np.sqrt(np.square(xs[:, None] - xs[None, :]).sum(axis=-1))

In [37]:
func = pdist_numpy
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 squareform(func(xs))

True
7.18 ms ± 1.55 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## JIT with `numba`

We use the `numba.jit` decorator which will trigger generation and execution of compiled code when the function is first called.

jit: Just-in-time compilation

In [38]:
from numba import jit

### Using `jit` as a function

In [39]:
pdist_numba_py = jit(pdist_py, nopython=True, cache=True)

In [40]:
func = pdist_numba_py
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
2.23 ms ± 48.2 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Using `jit` as a decorator

In [41]:
@jit(nopython=True, cache=True)
def pdist_numba_py_1(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            for k in range(p):
                A[i,j] += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(A[i,j])
    return A

In [42]:
func = pdist_numba_py_1
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
2.16 ms ± 57.3 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Can we make the code faster?

Note that in the inner loop, we are updating a matrix when we only need to update a scalar. Let's fix this.

In [43]:
@jit(nopython=True, cache=True)
def pdist_numba_py_2(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            d = 0.0
            for k in range(p):
                d += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(d)
    return A

In [44]:
func = pdist_numba_py_2
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
1.16 ms ± 173 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Can we make the code even faster?

We can also try to exploit symmetry.

In [45]:
@jit(nopython=True, cache=True)
def pdist_numba_py_sym(xs):
    """Unvectorized Python."""
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += (xs[i, k] - xs[j, k])**2
            A[i,j] = np.sqrt(d)
    A += A.T
    return A

In [46]:
func = pdist_numba_py_sym
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
564 µs ± 54.7 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Does `jit` work with vectorized code?

In [47]:
pdist_numba_vec = jit(pdist_vec, nopython=True, cache=True)

Only inner loop vectorized

In [48]:
%timeit -r3 -n3 pdist_vec(xs)

57.2 ms ± 1.07 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [49]:
func = pdist_numba_vec
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
1.73 ms ± 171 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Does `jit` work with broadcasting?

In [50]:
pdist_numba_numpy = jit(pdist_numpy, nopython=True, cache=True)

In [51]:
%timeit -r3 -n3 pdist_numpy(xs)

5.84 ms ± 940 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


This raises an error because `numba` does not know how to deal with dummy axes.

In [ ]:
try:
    %timeit -r3 -n3 pdist_numba_numpy(xs)
except Exception as e:
    print('Exception raised')

#### We need to use `reshape` to broadcast

In [52]:
def pdist_numpy_(xs):
    """Fully vectroized version."""
    return np.sqrt(np.square(xs.reshape(n,1,p) - xs.reshape(1,n,p)).sum(axis=-1))

In [53]:
pdist_numba_numpy_ = jit(pdist_numpy_, nopython=True, cache=True)

In [54]:
%timeit -r3 -n3 pdist_numpy_(xs)

5.61 ms ± 531 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [55]:
func = pdist_numba_numpy_
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
8.73 ms ± 136 µs per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Summary

- `numba` appears to work best when converting fairly explicit Python code
- This might change in the future as the `numba` JIT compiler becomes more sophisticated
- Always check optimized code for correctness
- We can use `timeit` magic as a simple way to benchmark functions

## Cython

Cython is an Ahead Of Time (AOT) compiler. It compiles the code and replaces the function invoked with the compiled version.

In the notebook, calling `%cython -a` magic shows code colored by how many Python C API calls are being made. You want to reduce the yellow as much as possible.

In [60]:
%%cython -a 

import numpy as np

def pdist_cython_1(xs):   
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += (xs[i,k] - xs[j,k])**2
            A[i,j] = np.sqrt(d)
    A += A.T
    return A

In [61]:
def pdist_base(xs):   
    n, p = xs.shape
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += (xs[i,k] - xs[j,k])**2
            A[i,j] = np.sqrt(d)
    A += A.T
    return A

In [62]:
%timeit -r3 -n3 pdist_base(xs)

518 ms ± 1.13 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [63]:
func = pdist_cython_1
print(np.allclose(func(xs), sol))
%timeit -r3 -n3 func(xs)

True
536 ms ± 3.51 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Cython with static types

- We provide types for all variables so that Cython can optimize their compilation to C code.
- Note `numpy` functions are optimized for working with `ndarrays` and have unnecessary overhead for scalars. We therefor replace them with math functions from the C `math` library.

`@cython.` decorators don't have a huge impact and aren't necessary

In [58]:
%%cython -a 

import cython
import numpy as np
cimport numpy as np
from libc.math cimport sqrt, pow

@cython.boundscheck(False)
@cython.wraparound(False)
def pdist_cython_2(double[:, :] xs):
    cdef int n, p
    cdef int i, j, k
    cdef double[:, :] A
    cdef double d
    
    n = xs.shape[0]
    p = xs.shape[1]
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            d = 0.0
            for k in range(p):
                d += pow(xs[i,k] - xs[j,k],2)
            A[i,j] = sqrt(d)
    for i in range(1, n):
        for j in range(i):
            A[i, j] = A[j, i]            
    return A

In [59]:
func = pdist_cython_2
print(np.allclose(func(xs), sol))
%timeit -r3 -n1 func(xs)

True
597 µs ± 8.98 µs per loop (mean ± std. dev. of 3 runs, 1 loop each)


`numba` is typically easier to use, but `cython` is more flexible